<a href="https://colab.research.google.com/github/Gooainekorea/test/blob/master/Rock_Paper_Scissor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio tensorflow==2.12.0 kagglehub

import gradio as gr
import tensorflow as tf
import numpy as np
from keras.models import load_model
from PIL import Image, ImageOps
import kagglehub
import os

# Kaggle 데이터셋 다운로드
dataset_path = kagglehub.dataset_download("sanikamal/rock-paper-scissors-dataset")
print("Path to dataset files:", dataset_path)

# 모델 파일 경로 설정 (사용자 환경에 맞게 수정 필요)
model_path = "/content/sample_data/keras_model.h5"
labels_path = "/content/sample_data/labels.txt"

# 모델 로드
try:
    model = load_model(model_path, compile=False)
except Exception as e:
    print(f"모델 로딩 중 오류 발생: {e}")
    exit()

# 클래스 레이블 로드
try:
    with open(labels_path, "r") as file:
        class_names = [line.strip() for line in file.readlines()]
except FileNotFoundError:
    print("labels.txt 파일을 찾을 수 없습니다.")
    exit()

# 예측 함수
def predict_rps(image):
    image = image.resize((224, 224))  # 모델 입력 크기 조정
    image = np.array(image) / 127.5 - 1  # 정규화
    image = np.expand_dims(image, axis=0)  # 배치 차원 추가

    predictions = model.predict(image)  # 예측 수행
    predicted_class = np.argmax(predictions)  # 가장 높은 확률의 클래스 선택
    confidence = predictions[0][predicted_class]  # 해당 클래스의 확률

    return f"{class_names[predicted_class]} (확률: {confidence:.2%})"

# Gradio 인터페이스 설정
interface = gr.Interface(
    fn=predict_rps,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="가위바위보 인식기",
    description="이미지를 업로드하면 가위, 바위, 보 중 하나를 예측합니다.",
)

# 실행
if __name__ == "__main__":
    interface.launch()
